# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

import geoviews

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

city_data_df.set_index("City_ID")

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,shakawe,-18.3667,21.8500,26.61,50,15,2.65,BW,1673507475
1,wanning,18.8003,110.3967,25.91,75,18,3.27,CN,1673507475
2,torbay,47.6666,-52.7314,-7.50,87,34,0.45,CA,1673507476
3,yellowknife,62.4560,-114.3525,-15.80,84,100,4.92,CA,1673507279
4,thompson,55.7435,-97.8558,-20.79,99,15,1.86,CA,1673507476
...,...,...,...,...,...,...,...,...,...
587,richards bay,-28.7830,32.0377,26.55,57,0,3.49,ZA,1673507711
588,sterling,39.0062,-77.4286,5.06,90,100,1.54,US,1673507711
589,hualmay,-11.0964,-77.6139,20.39,79,100,3.13,PE,1673507712


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500,size = "Humidity", color = "City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
filtered_city_data_df = city_data_df.loc[(city_data_df["Wind Speed"]<4.5)]
filtered_city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] ==0)]

# Drop any rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Display sample data
filtered_city_data_df.head()
filtered_city_data_df.set_index("City_ID")

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
8,sitka,57.0531,-135.3300,3.94,87,0,3.09,US,1673507478
16,ladnun,27.6500,74.3833,23.65,22,0,2.24,IN,1673507481
18,cidreira,-30.1811,-50.2056,21.54,84,0,4.60,BR,1673507482
21,praia,14.9215,-23.5087,21.30,78,0,8.75,CV,1673507483
23,atar,20.5169,-13.0499,16.54,14,0,3.70,MR,1673507484
...,...,...,...,...,...,...,...,...,...
580,makaha,21.4694,-158.2175,22.95,72,0,2.06,US,1673507709
581,kensington,-33.9167,151.2167,26.84,60,0,7.72,AU,1673507709
584,ostersund,63.1792,14.6357,2.25,80,0,6.17,SE,1673507710


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
8,sitka,US,57.0531,-135.3300,87,
16,ladnun,IN,27.6500,74.3833,22,
18,cidreira,BR,-30.1811,-50.2056,84,
21,praia,CV,14.9215,-23.5087,78,
23,atar,MR,20.5169,-13.0499,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {"categories": categories, "apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filters"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    json_address = name_address.json()
#     print(json_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = json_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
sitka - nearest hotel: Aspen Suites Hotel
ladnun - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
praia - nearest hotel: Casa Sodadi
atar - nearest hotel: فندق سكليل
barrow - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
geraldton - nearest hotel: Ocean Centre Hotel
punta arenas - nearest hotel: Hotel Lacolet
mar del plata - nearest hotel: Nuevo Ostende
seoul - nearest hotel: 뉴국제호텔
isla vista - nearest hotel: The Club & Guest House
busselton - nearest hotel: Broadwater Beach Resort
carnarvon - nearest hotel: No hotel found
baft - nearest hotel: هتل 4 ستاره در حال ساخت بام بافت
stephenville - nearest hotel: No hotel found
san patricio - nearest hotel: Best Western Plus
byron bay - nearest hotel: Lord Byron Resort
kapaa - nearest hotel: Pono Kai Resort
pochutla - nearest hotel: Posada San Jose
saint george - nearest hotel: The Advenire, an Autograph Collection Hotel
cabo san lucas - nearest hotel: Comfort Ro

,City,Country,Lat,Lng,Humidity,Hotel Name
8,sitka,US,57.0531,-135.3300,87,Aspen Suites Hotel
16,ladnun,IN,27.6500,74.3833,22,No hotel found
18,cidreira,BR,-30.1811,-50.2056,84,Hotel Castelo
21,praia,CV,14.9215,-23.5087,78,Casa Sodadi
23,atar,MR,20.5169,-13.0499,14,فندق سكليل


In [17]:
# Set index as city 
hotel_index_df = hotel_df.set_index("City")

# show reset df sample
hotel_index_df.head()


,Country,Lat,Lng,Humidity,Hotel Name
City,,,,,
sitka,US,57.0531,-135.3300,87,Aspen Suites Hotel
ladnun,IN,27.6500,74.3833,22,No hotel found
cidreira,BR,-30.1811,-50.2056,84,Hotel Castelo
praia,CV,14.9215,-23.5087,78,Casa Sodadi
atar,MR,20.5169,-13.0499,14,فندق سكليل


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_index_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500, hover_cols="all")

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)